# **Language Model**

In [1]:
import os

import torch

### **Data utils**

**Dictionary**: 

- A dictionary which maps all words and its index (word2idx-{word:idx}; idx2word-{idx:word}) 

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        
    def add_word(self, word):
        if not word in self.word2idx:
            # word not in the dict
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __len__(self):
        return len(self.word2idx)

**Corpus**
- Generate the content in `Dictionary`
- Tokenize the whole corpus

In [9]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
        
    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0  # the number of words in the corpus (different from vocab_size, which is unique word num)
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
                    
        # Tokenize the file content
        ids = torch.LongTensor(tokens)  # Create a list with size (tokens, ), all zero, token
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    # 一个huge list, 将corpus中每个句子的每个单词，用其对应的idx表示
                    ids[token] = self.dictionary.word2idx[word] 
                    token += 1
        num_batches = ids.size(0) // batch_size  # 将corpus中的单词划分为多个batch
        ids = ids[:num_batches * batch_size]  # 去除多余的最后一个batch的文本
        return ids.view(batch_size, -1)  # 将corpus中的文本，作为一个batch的数据

### **Main**

In [5]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

**Device**

In [6]:
# Device Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Hyper-parameter**

In [7]:
# Hyper-parameter
batch_size = 20
learning_rate = 0.002
num_epochs = 5

seq_length = 30
embed_size = 128
hidden_size = 1024
num_samples = 1000  # number of words to be sampled
num_layers = 1

**Load "Penn Treebank dataset"**

In [11]:
corpus = Corpus()
ids = corpus.get_data('data/02-LanguageModel/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [21]:
ids.size()

torch.Size([20, 46479])

In [15]:
num_batches

1549

**RNN based language model**

In [22]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed the word ids to vector
        x = self.embed(x)
        
        # Forward propagate
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output
        out = out.reshape(-1, out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [24]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

**Loss & Optimizer**

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Train the model**

In [27]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states]

# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)  # 用前面的词，预测后一个词 （batch_size, seq_len)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device) # 最后一个词 （batch_size, seq_len)

        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))


Epoch [1/5], Step[0/1549], Loss: 9.2122, Perplexity: 10018.34
Epoch [1/5], Step[100/1549], Loss: 6.0132, Perplexity: 408.80
Epoch [1/5], Step[200/1549], Loss: 5.9409, Perplexity: 380.27
Epoch [1/5], Step[300/1549], Loss: 5.7550, Perplexity: 315.78
Epoch [1/5], Step[400/1549], Loss: 5.6825, Perplexity: 293.69
Epoch [1/5], Step[500/1549], Loss: 5.1260, Perplexity: 168.34
Epoch [1/5], Step[600/1549], Loss: 5.1548, Perplexity: 173.26
Epoch [1/5], Step[700/1549], Loss: 5.3420, Perplexity: 208.92
Epoch [1/5], Step[800/1549], Loss: 5.2076, Perplexity: 182.65
Epoch [1/5], Step[900/1549], Loss: 5.0553, Perplexity: 156.85
Epoch [1/5], Step[1000/1549], Loss: 5.0837, Perplexity: 161.37
Epoch [1/5], Step[1100/1549], Loss: 5.3192, Perplexity: 204.21
Epoch [1/5], Step[1200/1549], Loss: 5.2172, Perplexity: 184.41
Epoch [1/5], Step[1300/1549], Loss: 5.0861, Perplexity: 161.77
Epoch [1/5], Step[1400/1549], Loss: 4.8080, Perplexity: 122.49
Epoch [1/5], Step[1500/1549], Loss: 5.1644, Perplexity: 174.94
Ep

**Test**

In [ ]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

In [28]:
# Save the model checkpoints
torch.save(model.state_dict(), './model/RNNLM.ckpt')